In [1]:
%store -r word_df
%store -r exp1_df

import pandas as pd
import numpy as np

# pickle.dump(word_df, fileObject)
# b = pickle.load(fileObject)

print(exp1_df.head())
print(word_df.head())



   title_char_count  title_word_count  title_word_density  \
0                39                 7            4.875000   
1                35                 8            3.888889   
2                33                 6            4.714286   
3                68                10            6.181818   
4                51                 8            5.666667   

   title_punctuation_count  title_title_word_count  \
0                        1                       4   
1                        1                       2   
2                        1                       2   
3                        4                       3   
4                        4                       2   

   title_upper_case_word_count  title_stopwords_count  desc_char_count  \
0                            1                      0             2589   
1                            0                      4              535   
2                            0                      2              293   
3           

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(exp1_df[['title_char_count', 'title_word_count', 'title_word_density', 'title_punctuation_count',
                                                             'title_title_word_count', 'title_upper_case_word_count', 'desc_char_count', 'desc_word_count',
                                                             'desc_word_density', 'desc_punctuation_count', 'desc_title_word_count', 'desc_upper_case_word_count',
                                                             'tags_char_count', 'tags_word_count', 'tags_word_density', 'tags_title_word_count', 'desc_upper_case_word_count']], 
                                                             word_df['target'], test_size=0.1, random_state=42)



In [3]:
# Train a logistic regression model
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=1, random_state=42)

In [4]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [5]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.33


In [6]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import metrics

print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.3333333333333333
[[28  2  0  0  0  0]
 [17  2  2  0  0  0]
 [13  4  3  0  1  0]
 [11  0  0  0  2  0]
 [ 6  0  0  0  0  0]
 [ 7  0  0  0  1  0]]
              precision    recall  f1-score   support

           0       0.34      0.93      0.50        30
           1       0.25      0.10      0.14        21
           2       0.60      0.14      0.23        21
           3       0.00      0.00      0.00        13
           4       0.00      0.00      0.00         6
           5       0.00      0.00      0.00         8

   micro avg       0.33      0.33      0.33        99
   macro avg       0.20      0.20      0.14        99
weighted avg       0.28      0.33      0.23        99



C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [7]:

print('Unimpressive performance from these features.  Moving on to experiment 2: count vectorizer and tfidf')


Unimpressive performance from these features.  Moving on to experiment 2: count vectorizer and tfidf


In [8]:
# scikit learn is particular about dimension handling so create one single column data frame with all words
single_df = pd.DataFrame()
single_df['words'] = word_df['search_word'] + ' ' + word_df["title"] + ' ' + word_df["description"] + ' ' + word_df['tags']
print(single_df.head(5))

                                               words
0  Siemens 15 Things You Didn't Know About SIEMEN...
1  Siemens What is it like to work at Siemens? Fr...
2  Siemens Siemens - More than just business Are ...
3  Siemens Siemens presents: The first 1,100 kV H...
4  Siemens Ce ??nseamnŽŸ sŽŸ lucrezi la Siemens -...


In [9]:
single_df['target'] = word_df['target']


In [10]:
X_train, X_test, y_train, y_test = train_test_split(single_df['words'], single_df['target'], test_size=0.1, random_state=42)
print(X_train.head())
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


559    iphone Quel iPhone choisir ? Fin 2018 et De?b...
748    mars 2030 …??…? …??…?? …??…?? …??……?ø…?? …?...
360    sports beIN SPORTS HABER CanlŽñ YayŽñnŽñ ???§?...
708    mars What will NASA's InSight do on Mars? The ...
596    iphone NEW iPhone Xr & Xs Max Models Hands On!...
Name: words, dtype: object
(99,)
(890,)
(99,)


In [11]:
# Utilize pipeline to make this process easier going forward
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer

tfidf_logreg = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('logreg', LogisticRegression(C=1, random_state=42))
                    ])


tfidf_logreg.fit(X_train, y_train)

y_pred = tfidf_logreg.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.40404040404040403
[[30  0  0  0  0  0]
 [19  0  2  0  0  0]
 [13  0  8  0  0  0]
 [10  1  1  0  1  0]
 [ 5  0  0  0  1  0]
 [ 5  1  0  0  1  1]]
              precision    recall  f1-score   support

           0       0.37      1.00      0.54        30
           1       0.00      0.00      0.00        21
           2       0.73      0.38      0.50        21
           3       0.00      0.00      0.00        13
           4       0.33      0.17      0.22         6
           5       1.00      0.12      0.22         8

   micro avg       0.40      0.40      0.40        99
   macro avg       0.40      0.28      0.25        99
weighted avg       0.37      0.40      0.30        99



C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv_logreg = Pipeline([
                     ('cv', CountVectorizer()),
                     ('logreg', LogisticRegression(C=1, random_state=42))
                    ])
cv_logreg.fit(X_train, y_train)

y_pred = tfidf_logreg.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.40404040404040403
[[30  0  0  0  0  0]
 [19  0  2  0  0  0]
 [13  0  8  0  0  0]
 [10  1  1  0  1  0]
 [ 5  0  0  0  1  0]
 [ 5  1  0  0  1  1]]
              precision    recall  f1-score   support

           0       0.37      1.00      0.54        30
           1       0.00      0.00      0.00        21
           2       0.73      0.38      0.50        21
           3       0.00      0.00      0.00        13
           4       0.33      0.17      0.22         6
           5       1.00      0.12      0.22         8

   micro avg       0.40      0.40      0.40        99
   macro avg       0.40      0.28      0.25        99
weighted avg       0.37      0.40      0.30        99



C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
print('This model is good at predicting non-ranked videos or basically subpar view count videos.')

This model is good at predicting non-ranked videos or basically subpar view count videos.


In [14]:
from sklearn.naive_bayes import MultinomialNB
tfidf_MNB = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB())
                    ])
tfidf_MNB.fit(X_train, y_train)

y_pred = tfidf_MNB.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))



0.32323232323232326
[[30  0  0  0  0  0]
 [21  0  0  0  0  0]
 [19  0  2  0  0  0]
 [13  0  0  0  0  0]
 [ 6  0  0  0  0  0]
 [ 8  0  0  0  0  0]]
              precision    recall  f1-score   support

           0       0.31      1.00      0.47        30
           1       0.00      0.00      0.00        21
           2       1.00      0.10      0.17        21
           3       0.00      0.00      0.00        13
           4       0.00      0.00      0.00         6
           5       0.00      0.00      0.00         8

   micro avg       0.32      0.32      0.32        99
   macro avg       0.22      0.18      0.11        99
weighted avg       0.31      0.32      0.18        99



C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
print('Accuracy is slightly worse than count based features Log Reg model')

Accuracy is slightly worse than count based features Log Reg model


In [16]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([
                       ('tfidf', TfidfVectorizer()),
                       ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                         alpha=1e-3, n_iter=5, random_state=42)),
                       ])
text_clf_svm.fit(X_train, y_train)

print(metrics.accuracy_score(y_test, y_pred))

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.32323232323232326
[[30  0  0  0  0  0]
 [21  0  0  0  0  0]
 [19  0  2  0  0  0]
 [13  0  0  0  0  0]
 [ 6  0  0  0  0  0]
 [ 8  0  0  0  0  0]]
              precision    recall  f1-score   support

           0       0.31      1.00      0.47        30
           1       0.00      0.00      0.00        21
           2       1.00      0.10      0.17        21
           3       0.00      0.00      0.00        13
           4       0.00      0.00      0.00         6
           5       0.00      0.00      0.00         8

   micro avg       0.32      0.32      0.32        99
   macro avg       0.22      0.18      0.11        99
weighted avg       0.31      0.32      0.18        99



C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
print('Accuracy is same as LogReg and better than MNB.')

Accuracy is same as LogReg and better than MNB.


In [18]:
print('Experiment w/ SGD:  alpha range')

# Create the list of alphas: alphas
alphas = np.arange(1,2,.1)

# Define train_and_predict()
def train_and_predict(alpha):
    SGD = Pipeline([
                       ('tfidf', TfidfVectorizer()),
                       ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                         alpha=alpha, n_iter=5, random_state=42)),
                       ])
    # Fit to the training data
    SGD.fit(X_train, y_train)
    # Compute accuracy: score
    score = metrics.accuracy_score(y_test, y_pred)
    return score

for alpha in alphas:
    print('Alpha: ', alpha)
    print('Score: ', train_and_predict(alpha))
    print()

Experiment w/ SGD:  alpha range
Alpha:  1.0


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Score:  0.32323232323232326

Alpha:  1.1


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Score:  0.32323232323232326

Alpha:  1.2000000000000002


...

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Score:  0.32323232323232326

Alpha:  1.8000000000000007


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Score:  0.32323232323232326

Alpha:  1.9000000000000008
Score:  0.32323232323232326



C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [19]:
print('No changes in accuracy based on different alpha learning rate')

No changes in accuracy based on different alpha learning rate


In [20]:
print('Experiment w/ SGD:  n_iter')

# Create the list of alphas: alphas
n_inters = np.arange(1,10,1)

# Define train_and_predict()
def train_and_predict(n_iter):
    SGD = Pipeline([
                       ('tfidf', TfidfVectorizer()),
                       ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                         alpha=alpha, n_iter=n_iter, random_state=42)),
                       ])
    # Fit to the training data
    SGD.fit(X_train, y_train)
    # Compute accuracy: score
    score = metrics.accuracy_score(y_test, y_pred)
    return score

for n_iter in n_inters:
    print('n_iter: ', n_iter)
    print('Score: ', train_and_predict(n_iter))
    print()

Experiment w/ SGD:  n_iter
n_iter:  1


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Score:  0.32323232323232326

n_iter:  2


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Score:  0.32323232323232326

n_iter:  3


...

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Score:  0.32323232323232326

n_iter:  8


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Score:  0.32323232323232326

n_iter:  9
Score:  0.32323232323232326



C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [21]:
print('No changes in accuracy based on testing number of iterations from 1 to 10')

No changes in accuracy based on testing number of iterations from 1 to 10


In [22]:
# Experiment with KNN: number of neighbors from 1 to 20
from sklearn.neighbors import KNeighborsClassifier 


nns = np.arange(1,20,1)
def train_and_predict(nn):
    text_knn = Pipeline([
                        ('tfidf', TfidfVectorizer()),
                        ('knn', KNeighborsClassifier(n_neighbors=nn) )
                        ])
    text_knn.fit(X_train, y_train)
    score = metrics.accuracy_score(y_test, y_pred)
    return score



for nn in nns:
    print('nn: ', nn)
    print('Score: ', train_and_predict(nn))
    print()

nn:  1
Score:  0.32323232323232326

nn:  2
Score:  0.32323232323232326

nn:  3
Score:  0.32323232323232326

nn:  4
Score:  0.32323232323232326

nn:  5
Score:  0.32323232323232326

nn:  6
Score:  0.32323232323232326

nn:  7
Score:  0.32323232323232326

nn:  8
Score:  0.32323232323232326

nn:  9
Score:  0.32323232323232326

nn:  10
Score:  0.32323232323232326

nn:  11
Score:  0.32323232323232326

nn:  12
Score:  0.32323232323232326

nn:  13
Score:  0.32323232323232326

nn:  14
Score:  0.32323232323232326

nn:  15
Score:  0.32323232323232326

nn:  16
Score:  0.32323232323232326

nn:  17
Score:  0.32323232323232326

nn:  18
Score:  0.32323232323232326

nn:  19
Score:  0.32323232323232326



In [23]:
print('No improvements with KNN and multiple neighbors ')

No improvements with KNN and multiple neighbors 


In [24]:
# Try XGBoost
from xgboost import XGBClassifier

text_xgb = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('xgb', XGBClassifier(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)) 
                    ])
text_xgb = text_xgb.fit(X_train, y_train)

predicted = text_xgb.predict(X_test)
print('Accuracy of initial XGBoost: ' + str(np.mean(predicted == y_test)))
print(predicted)


Accuracy of initial XGBoost: 0.3333333333333333
[4 0 0 2 0 0 2 0 0 0 0 1 0 0 0 0 0 0 2 2 0 0 0 0 0 0 0 2 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 4 0 0 4 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0
 0 4 0 0 4 0 1 5 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [25]:
# Try Random Forest initially without parameter tuning 
from sklearn.ensemble import RandomForestClassifier

text_rf = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('rf', RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)) 
                    ])
text_rf = text_rf.fit(X_train, y_train)

predicted = text_rf.predict(X_test)
print('Accuracy of initial Random Forest: ' + str(np.mean(predicted == y_test)))

Accuracy of initial Random Forest: 0.31313131313131315


In [26]:
# Create the basic token pattern
TOKENS_BASIC = '\\S+(?=\\s+)'

# Create the alphanumeric token pattern
TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'

# Instantiate basic CountVectorizer: vec_basic
vec_basic = TfidfVectorizer(token_pattern=TOKENS_BASIC)

# Instantiate alphanumeric CountVectorizer: vec_alphanumeric
vec_alphanumeric = TfidfVectorizer(token_pattern=TOKENS_ALPHANUMERIC)

# Create the text vector
text_vector = single_df['words']

# Fit and transform vec_basic - tokenizing using any non-whitespace characters
vec_basic.fit_transform(text_vector)

# Print number of tokens of vec_basic
print("There are {} tokens in the dataset".format(len(vec_basic.get_feature_names())))
print(vec_basic.get_feature_names())

There are 41585 tokens in the dataset
['!', '!!', '!!!', '!!!!ƒ??business', '!ayy', '!draw', '!gootime', '!hammer', '!how', '!tranquility', '!what', '!when', '!where', '!who', '!why', '"', '"*":', '".', '"......."', '"1"', '"10', '"10+1', '"13', '"2009"', '"2009,"', '"20off"', '"4k', '"???????æ?¯??¥?¥?¥\x81¥\x8f"', '"??????fenster\'s', '"??????monkeys', '"a', '"africa\'s', '"album', '"alive"', '"all', '"also', '"always', '"america\'s', '"american', '"ancient', '"angels', '"anonymous.official.reply@gmail.com"', '"are', '"atom', '"attribution', '"auld', '"awesome', '"baby', '"back', '"baldi\'s', '"bane\'s', '"beginner\'s",', '"beh??lter', '"bent', '"best', '"better', '"beverly', '"bloomberg', '"born', '"bowser\'s', '"boys', '"brianna', '"bring', '"britain\'s', '"bucket', '"budget"', '"c\'est', '"can\'t', '"canon', '"castle', '"cat\'s', '"cbs', '"cello', '"chang\'e",', '"chaos"', '"chaosxsilencer"', '"chill', '"chris"', '"chupacobbler"', '"clean', '"closer', '"cohen\'s', '"come', '"comedy

In [27]:
# The basic vector is too noisy since we included punctuaction, symbols, etc. as tokens
X = vec_alphanumeric.fit_transform(text_vector)

# Print number of tokens of vec_alphanumeric
print("There are {} alpha-numeric tokens in the dataset".format(len(vec_alphanumeric.get_feature_names())))

#Create sparse matrix as X using only alphanumeric values
word_vector = X.toarray()
y = single_df['target']

%store word_vector

There are 17873 alpha-numeric tokens in the dataset
Stored 'word_vector' (ndarray)


In [28]:
X_train, X_test, y_train, y_test = train_test_split(word_vector, y, random_state=22)
print(X_train)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.06396486 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [29]:
# Again, try random search CV to search optimal hyperparameters
from sklearn.model_selection import RandomizedSearchCV
#Create a random grid for possible parameters to attempt and then use random search

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)



{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [30]:
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, scoring = 'neg_mean_squared_error', n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 13.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 26.4min finished
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring='neg_mean_squared_error

In [31]:
#Print and evaluate best parameters from the search
print(rf_random.best_params_)
best_random = rf_random.best_estimator_

{'n_estimators': 800, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 100, 'bootstrap': False}


In [32]:
#Evaluate performance of the base model w/ searched hyperparameters
random_model = RandomForestClassifier(n_estimators = 400, min_samples_split=2, min_samples_leaf=1, 
                                   max_features = 'sqrt', max_depth = None, bootstrap = False, random_state = 42)

random_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [33]:
#Evaluate performance of random forest with random searched hyperparameters
from sklearn.metrics import mean_squared_error

print('Random Forest R squared: %.4f' % random_model.score(X_test, y_test))
y_pred = random_model.predict(X_test)
forest_mse = mean_squared_error(y_pred, y_test)
forest_rmse = np.sqrt(forest_mse)
print('Random Forest RMSE: %.4f' % forest_rmse)
print(metrics.accuracy_score(y_test, y_pred))

Random Forest R squared: 0.4637
Random Forest RMSE: 1.8426
0.4637096774193548


In [34]:
print('We see a roughly 3% improvement with random searched parameters for RF')

We see a roughly 3% improvement with random searched parameters for RF


In [35]:
# Try N-grams as a feature with MNB   - Because we are passing data into pipeline for vectorization, pass in raw string,

X_train, X_test, y_train, y_test = train_test_split(single_df['words'], single_df['target'], test_size=0.1, random_state=42)


logregpipe = Pipeline([
                      ('vectorizer', TfidfVectorizer(min_df=1, stop_words='english', ngram_range=(1, 2))),
                     ('logreg', LogisticRegression(C=1))
                    ])
logreg = logregpipe.fit(X_train, y_train)
test_predicted = logreg.predict(X_test)
train_predicted = logreg.predict(X_train)

print('Accuracy of Log Reg train set: ' + str(np.mean(train_predicted == y_train)))
print('Accuracy of Log Reg test set: ' + str(np.mean(test_predicted == y_test)))


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy of Log Reg train set: 0.651685393258427
Accuracy of Log Reg test set: 0.40404040404040403


In [36]:
MNBpipe = Pipeline([
                     ('vectorizer', TfidfVectorizer(min_df=1, stop_words='english', ngram_range=(1, 2))),
                     ('clf', MultinomialNB())
                    ])

MNB = MNBpipe.fit(X_train, y_train)
test_predicted = MNB.predict(X_test)
train_predicted = MNB.predict(X_train)

print('Accuracy of MNB train set: ' + str(np.mean(train_predicted == y_train)))
print('Accuracy of MNB test set: ' + str(np.mean(test_predicted == y_test)))


Accuracy of MNB train set: 0.49887640449438203
Accuracy of MNB test set: 0.35353535353535354


In [37]:
from nltk.corpus import stopwords
import nltk


In [38]:
from nltk.tokenize import sent_tokenize, word_tokenize
tokenized_words = single_df['words'].apply(word_tokenize) 
tokenized_words = pd.DataFrame(tokenized_words).reset_index()
tokenized_words.columns = ['index','unstemmed']
tokenized_words = tokenized_words.rename_axis(None)
print(tokenized_words.head())

   index                                          unstemmed
0      0  [Siemens, 15, Things, You, Did, n't, Know, Abo...
1      1  [Siemens, What, is, it, like, to, work, at, Si...
2      2  [Siemens, Siemens, -, More, than, just, busine...
3      3  [Siemens, Siemens, presents, :, The, first, 1,...
4      4  [Siemens, Ce, ?, ?, nseamnŽŸ, sŽŸ, lucrezi, la...


In [39]:
stop_words = set(stopwords.words('english'))
# filtered_sentetest['tweet'].apply(lambda x: [item for item in x if item not in stop])nce = tokenized_words.apply(lambda x : [w for w in x if w.lower() not in stop_words]) 
#filtered_sentence = [w for w in tokenized_words['unstemmed'] if not w in stop_words] 
no_stops = tokenized_words['unstemmed'].apply(lambda x: ' '.join([item for item in x if item not in stop_words]))
print(no_stops)

0      Siemens 15 Things You Did n't Know About SIEME...
1      Siemens What like work Siemens ? From digitali...
2      Siemens Siemens - More business Are interested...
3      Siemens Siemens presents : The first 1,100 kV ...
4      Siemens Ce ? ? nseamnŽŸ sŽŸ lucrezi la Siemens...
5      Siemens Siemens SPS IPC Drives 2018 - Introduc...
6      Siemens The Siemens SGT-800 A 50-MW-class indu...
7      Siemens Siemens freezes $ 20B deal Riyadh | Mo...
8      Siemens Siemens SL45 : ? ¨ ? æ¥ ? ? ý¥ ? ? ? M...
9      Siemens Siemens SGT-750 gas turbine flythrough...
10     Siemens Siemens glance - Digitalization Our wo...
11     Siemens Siemens CT 636 les1 Witam je¬eli inte...
12     Siemens Siemens GA400 ƒ ? ? World Class Design...
13     Siemens Siemens Since purchased Westinghouse '...
14     Siemens Siemens presents : The first 1,100 kV ...
15     Siemens Siemens CEO Kaeser Earnings , Business...
16     Siemens ? ® ? ­ ? ¯ ? ? ¥ ? ¥ ? ¥¥ ¥ ¥ ? ? ...
17     Siemens Waschmaschine Te

In [40]:
#Try Snowball Stemmer since this is more modern stemming method vs. PorterStemmer

import nltk
sno = nltk.stem.SnowballStemmer('english')
stemmed_words = pd.DataFrame()

In [41]:
stemmed_words['stemmed'] = tokenized_words["unstemmed"].apply(lambda x: ' '.join([sno.stem(y) for y in x]))
tokenized_words['stemmed'] = tokenized_words["unstemmed"].apply(lambda x: [sno.stem(y) for y in x])
print(tokenized_words['stemmed'])



0      [siemen, 15, thing, you, did, n't, know, about...
1      [siemen, what, is, it, like, to, work, at, sie...
2      [siemen, siemen, -, more, than, just, busi, ar...
3      [siemen, siemen, present, :, the, first, 1,100...
4      [siemen, ce, ?, ?, nseamnžÿ, sžÿ, lucrezi, la,...
5      [siemen, siemen, at, sps, ipc, drive, 2018, -,...
6      [siemen, the, siemen, sgt-800, a, 50-mw-class,...
7      [siemen, siemen, freez, $, 20b, deal, with, ri...
8      [siemen, siemen, sl45, :, ?, ¨, ?, æ¥, ?, ?, ý...
9      [siemen, siemen, sgt-750, gas, turbin, flythro...
10     [siemen, siemen, at, a, glanc, -, digit, our, ...
11     [siemen, siemen, ct, 636, les1, witam, je¬, i...
12     [siemen, siemen, ga400, ƒ, ?, ?, world, class,...
13     [siemen, siemen, sinc, it, purchas, westinghou...
14     [siemen, siemen, present, :, the, first, 1,100...
15     [siemen, siemen, ceo, kaeser, on, earn, ,, bus...
16     [siemen, ?, ®, ?, ­, ?, ¯, ?, ?, ¥, ?, ¥, ?, ¥...
17     [siemen, waschmaschin, t

In [65]:
#1-2 gram with stemmed words

X_train, X_test, y_train, y_test = train_test_split(stemmed_words['stemmed'], single_df['target'], test_size=0.1, random_state=42)


logregpipe = Pipeline([
                      ('vectorizer', TfidfVectorizer(min_df=1, stop_words='english', ngram_range=(1, 2))),
                     ('logreg', LogisticRegression(C=1))
                    ])
logreg = logregpipe.fit(X_train, y_train)
test_predicted = logreg.predict(X_test)
train_predicted = logreg.predict(X_train)

print(metrics.accuracy_score(y_test, test_predicted))
print('Accuracy of Log Reg train set: ' + str(np.mean(train_predicted == y_train)))
print('Accuracy of Log Reg test set: ' + str(np.mean(test_predicted == y_test)))

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.36363636363636365
Accuracy of Log Reg train set: 0.6359550561797753
Accuracy of Log Reg test set: 0.36363636363636365


In [64]:
xgb_pipe = Pipeline([
                      ('vectorizer', TfidfVectorizer(min_df=1, stop_words='english', ngram_range=(1, 2))),
                     ('xgb', XGBClassifier(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10))
                    ])
xgb = xgb_pipe.fit(X_train, y_train)
test_predicted = xgb.predict(X_test)
train_predicted = xgb.predict(X_train)

print(metrics.accuracy_score(y_test, test_predicted))
print('Accuracy of Log Reg train set: ' + str(np.mean(train_predicted == y_train)))
print('Accuracy of Log Reg test set: ' + str(np.mean(test_predicted == y_test)))

0.37373737373737376
Accuracy of Log Reg train set: 0.6955056179775281
Accuracy of Log Reg test set: 0.37373737373737376


In [44]:

tfidf_logreg = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('logreg', LogisticRegression(C=1, random_state=42))
                    ])
tfidf_logreg.fit(X_train, y_train)

y_pred = tfidf_logreg.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.3939393939393939
[[29  0  0  0  1  0]
 [19  0  2  0  0  0]
 [13  0  8  0  0  0]
 [10  1  1  0  1  0]
 [ 5  0  0  0  1  0]
 [ 5  1  0  0  1  1]]
              precision    recall  f1-score   support

           0       0.36      0.97      0.52        30
           1       0.00      0.00      0.00        21
           2       0.73      0.38      0.50        21
           3       0.00      0.00      0.00        13
           4       0.25      0.17      0.20         6
           5       1.00      0.12      0.22         8

   micro avg       0.39      0.39      0.39        99
   macro avg       0.39      0.27      0.24        99
weighted avg       0.36      0.39      0.29        99



In [68]:


tfidf_logreg = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('rf',RandomForestClassifier(n_estimators = 400, min_samples_split=2, min_samples_leaf=1, 
                                   max_features = 'sqrt', max_depth = None, bootstrap = False, random_state = 42))
                    ])
tfidf_logreg.fit(X_train, y_train)

y_pred = tfidf_logreg.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.40404040404040403
[[28  1  0  0  1  0]
 [18  3  0  0  0  0]
 [13  0  6  0  2  0]
 [ 7  3  2  1  0  0]
 [ 5  0  0  0  1  0]
 [ 5  0  1  0  1  1]]
              precision    recall  f1-score   support

           0       0.37      0.93      0.53        30
           1       0.43      0.14      0.21        21
           2       0.67      0.29      0.40        21
           3       1.00      0.08      0.14        13
           4       0.20      0.17      0.18         6
           5       1.00      0.12      0.22         8

   micro avg       0.40      0.40      0.40        99
   macro avg       0.61      0.29      0.28        99
weighted avg       0.57      0.40      0.34        99



In [69]:
#Use stop words 

X_train, X_test, y_train, y_test = train_test_split(no_stops, single_df['target'], test_size=0.1, random_state=42)


In [46]:
xgb_pipe = Pipeline([
                      ('vectorizer', TfidfVectorizer(min_df=1, stop_words='english', ngram_range=(1, 2))),
                     ('logreg', LogisticRegression(C=1, random_state=42))
                    ])
xgb = xgb_pipe.fit(X_train, y_train)
test_predicted = xgb.predict(X_test)
train_predicted = xgb.predict(X_train)

print(metrics.accuracy_score(y_test, test_predicted))
print('Accuracy of Log Reg train set: ' + str(np.mean(train_predicted == y_train)))
print('Accuracy of Log Reg test set: ' + str(np.mean(test_predicted == y_test)))

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.40404040404040403
Accuracy of Log Reg train set: 0.648314606741573
Accuracy of Log Reg test set: 0.40404040404040403


In [47]:
log_pipe = Pipeline([
                      ('vectorizer', TfidfVectorizer(min_df=1, stop_words='english', ngram_range=(1, 2))),
                     ('xgb', XGBClassifier(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10))
                    ])
log = log_pipe.fit(X_train, y_train)
test_predicted = log.predict(X_test)
train_predicted = log.predict(X_train)

print(metrics.accuracy_score(y_test, test_predicted))
print('Accuracy of Log Reg train set: ' + str(np.mean(train_predicted == y_train)))
print('Accuracy of Log Reg test set: ' + str(np.mean(test_predicted == y_test)))

0.3838383838383838
Accuracy of Log Reg train set: 0.6921348314606741
Accuracy of Log Reg test set: 0.3838383838383838


In [48]:
X_train, X_test, y_train, y_test = train_test_split(single_df['words'], single_df['target'], test_size=0.1, random_state=42)
print(X_train.head())
print(X_test.shape)
print(y_train.head())

559    iphone Quel iPhone choisir ? Fin 2018 et De?b...
748    mars 2030 …??…? …??…?? …??…?? …??……?ø…?? …?...
360    sports beIN SPORTS HABER CanlŽñ YayŽñnŽñ ???§?...
708    mars What will NASA's InSight do on Mars? The ...
596    iphone NEW iPhone Xr & Xs Max Models Hands On!...
Name: words, dtype: object
(99,)
559    0
748    0
360    2
708    2
596    4
Name: target, dtype: int64


In [49]:
from gensim.models import Word2Vec


model = Word2Vec(tokenized_words['unstemmed'])


In [50]:
words = list(model.wv.vocab)
print(words)

['Siemens', '15', 'Things', 'You', 'Did', "n't", 'Know', 'About', 'SIEMENS', '|', 'SUBSCRIBE', 'to', ':', 'https', '?', 'sub_confirmation=1', '10', 'Most', 'Expensive', '//www.youtube.com/watch', 'Samsung', 'In', 'this', 'video', 'we', "'ll", 'try', 'answer', 'the', 'following', 'questions', 'Who', 'How', 'big', 'is', 'What', "'", 'worth', 'products', 'making', 'Why', 'one', 'of', 'most', 'top', 'companies', 'becoming', 'did', 'energy', 'business', 'any', 'records', 'best', 'product', 'WATCH', 'MORE', 'VIDEOS', 'ON', '!', '&', 'index=1', 'a', 'World', "'s", 'People', 'Travel', 't=2s', 'Dark', 'Celebrity', 'Videos', '-', 'Follow', 'us', 'on', 'INSTAGRAM', 'for', 'amazing', 'Do', 'miss', 'latest', 'News', 'only', 'Facebook', '--', 'largest', 'community', 'fine', 'living', 'in', 'world', '.', 'We', 'are', '#', '1', 'online', 'expensive', 'things', 'and', 'such', 'as', ',', 'many', 'more', 'destination', 'content', 'Our', 'website', 'social', 'network', 'people', 'who', 'about', 'Join', 't

In [51]:
stemmed_model = Word2Vec(tokenized_words['stemmed'])

In [52]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [53]:
# Averaging word vectors for all words in a text
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter (w2v.values())))

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [54]:
from collections import defaultdict

class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(next(iter (w2v.values())))

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [55]:
#unstemmed features, w2v, LogReg

logreg_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),
    ('logreg', LogisticRegression(C=1, random_state=42))
    ])
logreg_w2v_tfidf = Pipeline([
    ("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)),
    ('logreg', LogisticRegression(C=1, random_state=42))
    ])

In [56]:
#stemmed features, w2v, XGBclassifier

logreg_stemmed_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(stemmed_model)),
    ('xgb', XGBClassifier(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10))
    ])
logreg_stemmed_w2v_tfidf = Pipeline([
    ("word2vec vectorizer", TfidfEmbeddingVectorizer(stemmed_model)),
    ('xgb',XGBClassifier(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10))
    ])

In [57]:
logreg_w2v.fit(X_train, y_train)

y_pred = logreg_w2v.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))



0.30303030303030304


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [58]:


logreg_w2v_tfidf.fit(X_train, y_train)

y_pred = logreg_w2v_tfidf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.30303030303030304


In [59]:
logreg_stemmed_w2v.fit(X_train, y_train)

y_pred = logreg_stemmed_w2v.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


0.3838383838383838


In [60]:
logreg_stemmed_w2v_tfidf.fit(X_train, y_train)

y_pred = logreg_stemmed_w2v_tfidf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


0.41414141414141414


In [61]:

%store single_df

Stored 'single_df' (DataFrame)
